In [21]:
from dotenv import load_dotenv

load_dotenv()


True

In [22]:
from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini",
   # api_key=os.environ.get("OPENAI_API_KEY")
)

In [23]:
from autogen_core.models import UserMessage

result = await model_client.create(
    [UserMessage(content="Tell me about Elon Musk in 2 bullet points", source="user")]
    )
print(result.content)

- **Entrepreneur and Innovator**: Elon Musk is the CEO and lead designer of SpaceX, CEO and product architect of Tesla, Inc., and co-founder of companies such as Neuralink and The Boring Company, focusing on advancing space exploration, electric vehicles, and brain-computer interfacing.

- **Visionary Goals**: He is known for his ambitious goals, including making life multi-planetary by colonizing Mars, accelerating the world's transition to sustainable energy, and developing technologies to integrate artificial intelligence with human brains.


In [18]:
import nest_asyncio
nest_asyncio.apply()


In [78]:
import sys
import asyncio

if sys.platform == "win32":
    asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())


In [20]:
import os
import asyncio
from autogen_ext.code_executors.local import LocalCommandLineCodeExecutor
from autogen_ext.tools.code_execution import PythonCodeExecutionTool
from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken
from autogen_agentchat.ui import Console


# Define project path
project_path = r"C:\Users\vasanth\Desktop\ai_deployer_project\test-repos\fastapi"

# Create Python code execution tool
python_tool = PythonCodeExecutionTool(LocalCommandLineCodeExecutor(
    work_dir=project_path
))


tech_stack_analysis_agent = AssistantAgent(
        "tech_stack_analysis_agent",
        description="you are linux assistant with python skills, answer any prompt asked",
        tools=[python_tool],  # Only pass execution tools
        model_client=model_client,
        system_message=f"""
        list the current directory structure using the given tool and give it in a tree like structure , use the given python tool to do it
        """,
        reflect_on_tool_use=True
    )

res = await tech_stack_analysis_agent.run(task="give me the current directory structure")


In [7]:
res.messages

[TextMessage(source='user', models_usage=None, metadata={}, content='give me the current directory structure', type='TextMessage'),
 ToolCallRequestEvent(source='tech_stack_analysis_agent', models_usage=RequestUsage(prompt_tokens=95, completion_tokens=128), metadata={}, content=[FunctionCall(id='call_Yrc6j2OjOB8J5XBgSMjUoETz', arguments='{"code":"import os\\n\\ndef list_directory_structure(path, level=0):\\n    indent = \'    \' * level\\n    structure = \'\'\\n    for item in os.listdir(path):\\n        full_path = os.path.join(path, item)\\n        structure += f\'{indent}{item}\\\\n\'\\n        if os.path.isdir(full_path):\\n            structure += list_directory_structure(full_path, level + 1)\\n    return structure\\n\\ncurrent_directory = os.getcwd()\\ndirectory_structure = list_directory_structure(current_directory)\\ndirectory_structure"}', name='CodeExecutor')], type='ToolCallRequestEvent'),
 ToolCallExecutionEvent(source='tech_stack_analysis_agent', models_usage=None, metada

In [ ]:
import venv
from pathlib import Path
import asyncio

from autogen_core import CancellationToken
from autogen_core.code_executor import CodeBlock
from autogen_ext.code_executors.local import LocalCommandLineCodeExecutor


async def setup_venv(work_dir):
    # Create project directory if not exists
    work_dir.mkdir(exist_ok=True)

    # Create virtual environment
    venv_dir = work_dir / ".venv"
    venv_builder = venv.EnvBuilder(with_pip=True)
    venv_builder.create(venv_dir)
    return venv_builder.ensure_directories(venv_dir)


async def execute_command(local_executor, command):
    # Execute shell command
    await local_executor.execute_code_blocks(
        code_blocks=[CodeBlock(language="bash", code=command)],
        cancellation_token=CancellationToken(),
    )


async def retrieve_folder_structure():
    work_dir = Path("project_structure")
    venv_context = await setup_venv(work_dir)

    local_executor = LocalCommandLineCodeExecutor(work_dir=work_dir, virtual_env_context=venv_context)

    # Install tree (if not installed) and list folder structure
    await execute_command(local_executor, "sudo apt-get install tree -y")  # For Linux users
    await execute_command(local_executor, "tree")


if __name__ == "__main__":
    asyncio.run(retrieve_folder_structure())


In [10]:
# correct one

import os
import asyncio
from autogen_ext.code_executors.local import LocalCommandLineCodeExecutor
from autogen_ext.tools.code_execution import PythonCodeExecutionTool
from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken
from autogen_agentchat.ui import Console

# Async function to get directory structure
async def get_tree(directory, prefix=""):
    """Recursively stores a directory structure like the tree command in a string."""
    tree_str = ""
    try:
        entries = sorted(os.listdir(directory))
    except FileNotFoundError:
        return f"Error: Directory '{directory}' not found."

    for index, entry in enumerate(entries):
        path = os.path.join(directory, entry)
        is_last = index == len(entries) - 1  # Check if it's the last entry
        
        connector = "└── " if is_last else "├── "
        tree_str += prefix + connector + entry + "\n"

        if os.path.isdir(path):
            extension = "    " if is_last else "│   "
            tree_str += await get_tree(path, prefix + extension)  # Recursively await
    # print(tree_str)
    return tree_str

# Async function to detect tech stack based on dependency files
async def detect_tech_stack(directory):
    """Analyzes dependency files to determine the tech stack."""
    stack_info = []
    detected_framework = "unknown"
    
    files = os.listdir(directory)

    if "requirements.txt" in files:
        stack_info.append("Python - Found requirements.txt")
        with open(os.path.join(directory, "requirements.txt"), "r") as f:
            requirements = f.read().lower()

            if "fastapi" in requirements:
                detected_framework = "FastAPI"
                print("=============fast found===================")
            elif "flask" in requirements:
                detected_framework = "Flask"
            elif "django" in requirements:
                detected_framework = "Django"

    elif "pyproject.toml" in files:
        stack_info.append("Python - Found pyproject.toml")
    elif "Pipfile" in files:
        stack_info.append("Python - Found Pipfile")
    elif "pom.xml" in files:
        stack_info.append("Java (Maven) - Found pom.xml")
    elif "build.gradle" in files:
        stack_info.append("Java (Gradle) - Found build.gradle")
    elif "package.json" in files:
        stack_info.append("JavaScript (Node.js) - Found package.json")

    if detected_framework != "unknown":
        stack_info.append(f"Detected Framework: {detected_framework}")

    return detected_framework, "\n".join(stack_info) if stack_info else "Tech stack could not be determined."

# Define project path
project_path = r"C:\Users\vasanth\Desktop\ai_deployer_project\test-repos\fastapi"

# Create Python code execution tool
python_tool = PythonCodeExecutionTool(LocalCommandLineCodeExecutor(
    work_dir=project_path
))

# Create agents
async def setup_agent():
    tree_structure = await get_tree(project_path)  # Get directory structure
    framework, tech_stack = await detect_tech_stack(project_path)  # Detect tech stack
    print(tech_stack)

    # Determine correct CMD based on framework
    # cmd_mapping = {
    #     "FastAPI": 'CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"]',
    #     "Flask": 'CMD ["python", "app.py"]',
    #     "Django": 'CMD ["gunicorn", "--bind", "0.0.0.0:8000", "project.wsgi:application"]'
    # }
    # framework_cmd = cmd_mapping.get(framework, 'CMD ["python", "app/main.py"]')

    tech_stack_analysis_agent = AssistantAgent(
        "tech_stack_analysis_agent",
        description="An agent that analyzes the tech stack and generates a Dockerfile.",
        tools=[python_tool],  # Only pass execution tools
        model_client=model_client,
        system_message=f"""
        You are a tech_stack_analysis_agent.
        Your job is to analyze the tech stack by finding dependency files like requirements.txt, pom.xml, package.json, etc.
        Check for frameworks like Flask, Django, or FastAPI.

        **Detected Tech Stack:**  
        {tech_stack}

        **Framework-detected:**  
        {framework}

        Based on this, generate a valid Dockerfile for the project.
        Your response should **only** be a valid Dockerfile with no explanations.

        **Project Structure:**  
        {tree_structure}
        """,
        reflect_on_tool_use=True
    )

    return tech_stack_analysis_agent

async def main():
    agent = await setup_agent()
    await Console(agent.run_stream(task="Create a Dockerfile for the given project."))


# Run the async function
await main()


=============fast found===================
Python - Found requirements.txt
Detected Framework: FastAPI
---------- user ----------
Create a Dockerfile for the given project.


C:\Users\vasanth\AppData\Local\Temp\ipykernel_12656\2601466659.py:74: UserWarning: The current event loop policy is not WindowsProactorEventLoopPolicy. This may cause issues with subprocesses. Try setting the event loop policy to WindowsProactorEventLoopPolicy. For example: `asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())`. See https://docs.python.org/3/library/asyncio-eventloop.html#asyncio.ProactorEventLoop.
  python_tool = PythonCodeExecutionTool(LocalCommandLineCodeExecutor(


---------- tech_stack_analysis_agent ----------
```
# Dockerfile

# Use the official Python image from the Docker Hub
FROM python:3.13

# Set the working directory in the container
WORKDIR /app

# Copy the requirements.txt file to the working directory
COPY requirements.txt .

# Install the dependencies from requirements.txt
RUN pip install --no-cache-dir -r requirements.txt

# Copy the rest of the application code to the working directory
COPY . .

# Expose the appropriate port (FastAPI default is 8000)
EXPOSE 8000

# Command to run the application using uvicorn
CMD ["uvicorn", "app.main:app", "--host", "0.0.0.0", "--port", "8000", "--reload"]
```


In [19]:
import asyncio
from autogen_ext.code_executors.local import LocalCommandLineCodeExecutor
from autogen_ext.tools.code_execution import PythonCodeExecutionTool, CodeExecutionInput
from autogen_core import CancellationToken

# Initialize the LocalCommandLineCodeExecutor
executor = LocalCommandLineCodeExecutor(work_dir=".")

# Wrap it inside the PythonCodeExecutionTool
python_tool = PythonCodeExecutionTool(executor)

# Sample Python code to execute
code = """
print("Hello from Local Executor!")
a = 10
b = 20
print("Sum:", a + b)
"""

# Async function to run the code
async def run_code():
    # Create a cancellation token
    token = CancellationToken()

    # Wrap the code inside CodeExecutionInput
    code_input = CodeExecutionInput(code=code)

    # Run the code with the cancellation token
    result = await python_tool.run(code_input, token)
    print("Execution Output:")
    print(result)

if __name__ == "__main__":
    # For Windows compatibility
    asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())
    asyncio.run(run_code())


C:\Users\vasanth\AppData\Local\Temp\ipykernel_12656\1470294700.py:7: UserWarning: The current event loop policy is not WindowsProactorEventLoopPolicy. This may cause issues with subprocesses. Try setting the event loop policy to WindowsProactorEventLoopPolicy. For example: `asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())`. See https://docs.python.org/3/library/asyncio-eventloop.html#asyncio.ProactorEventLoop.
  executor = LocalCommandLineCodeExecutor(work_dir=".")


NotImplementedError: 

In [ ]:
# langgraph code 


import os
import json
from typing import Dict, List, Any
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END

# Define a simple dict-based state
def create_initial_state(project_path: str) -> Dict:
    """Create the initial state dictionary."""
    return {
        "project_path": project_path,
        "tree_structure": "",
        "tech_stack": "",
        "framework": "",
        "framework_cmd": "",
        "docker_file": "",
        "messages": [],
        "project_info": {"files": [], "file_contents": {}}
    }

# Function to get directory structure
def get_tree(directory: str, prefix: str = "") -> str:
    """Recursively stores a directory structure like the tree command in a string."""
    tree_str = ""
    try:
        entries = sorted(os.listdir(directory))
    except FileNotFoundError:
        return f"Error: Directory '{directory}' not found."

    for index, entry in enumerate(entries):
        path = os.path.join(directory, entry)
        is_last = index == len(entries) - 1  # Check if it's the last entry
        
        connector = "└── " if is_last else "├── "
        tree_str += prefix + connector + entry + "\n"

        if os.path.isdir(path):
            extension = "    " if is_last else "│   "
            tree_str += get_tree(path, prefix + extension)

    return tree_str

# Function to gather file contents for analysis
def gather_project_info(directory: str) -> Dict[str, Any]:
    """Gathers key files and their contents for tech stack analysis."""
    project_info = {
        "files": [],
        "file_contents": {}
    }
    
    important_files = [
        "requirements.txt", "pyproject.toml", "Pipfile", 
        "package.json", "pom.xml", "build.gradle",
        "app.py", "main.py", "wsgi.py", "manage.py"
    ]
    
    for root, dirs, files in os.walk(directory):
        for file in files:
            rel_path = os.path.join(root, file).replace(directory, "").strip("/\\")
            project_info["files"].append(rel_path)
            
            if file in important_files:
                try:
                    file_path = os.path.join(root, file)
                    with open(file_path, "r", encoding="utf-8") as f:
                        content = f.read()
                        project_info["file_contents"][file] = content
                except Exception as e:
                    project_info["file_contents"][file] = f"Error reading file: {str(e)}"
    
    return project_info

# Node 1: Get project structure
def get_project_structure(state: Dict) -> Dict:
    """Gets the project structure and updates the state."""
    # Make a copy of the state to avoid modifying the input directly
    new_state = state.copy()
    
    # Get tree structure
    tree_structure = get_tree(new_state["project_path"])
    new_state["tree_structure"] = tree_structure
    
    # Get project info
    project_info = gather_project_info(new_state["project_path"])
    new_state["project_info"] = project_info
    
    return new_state

# Node 2: Analyze tech stack using LLM
def analyze_tech_stack(state: Dict) -> Dict:
    """Analyzes the tech stack using LLM and updates the state."""
    # Make a copy of the state
    new_state = state.copy()
    
    # Initialize LLM
    llm = ChatOpenAI(model_name="gpt-4")
    
    # Debug print
    print(f"Keys in state: {list(new_state.keys())}")
    print(f"Project info keys: {list(new_state['project_info'].keys())}")
    
    # Build file contents string
    file_contents_str = ""
    for file_name, content in new_state["project_info"]["file_contents"].items():
        # Truncate long file contents to prevent exceeding token limits
        content_preview = content[:500] + "..." if len(content) > 500 else content
        file_contents_str += f"\n--- {file_name} ---\n{content_preview}\n"
    
    prompt = f"""
    You are a tech stack analysis agent specialized in identifying technologies used in software projects.
    
    Analyze the following project structure and file contents to determine:
    1. The programming language(s) used
    2. The framework(s) used (e.g., Flask, FastAPI, Django for Python)
    3. The appropriate command to run the application in a Docker container
    
    Project Structure:
    {new_state["tree_structure"]}
    
    Key File Contents:
    {file_contents_str}
    
    Respond with a JSON structure with the following fields:
    {{
        "tech_stack": "Detailed description of detected technologies",
        "framework": "Main framework detected (if any)",
        "framework_cmd": "The appropriate Docker CMD command for running this application"
    }}
    """
    
    messages = [HumanMessage(content=prompt)]
    response = llm.invoke(messages)
    
    # Parse the response to extract tech stack, framework and command
    try:
        analysis_result = json.loads(response.content)
        new_state["tech_stack"] = analysis_result.get("tech_stack", "Unknown tech stack")
        new_state["framework"] = analysis_result.get("framework", "Unknown framework")
        new_state["framework_cmd"] = analysis_result.get("framework_cmd", 'CMD ["python", "app.py"]')
    except json.JSONDecodeError:
        # Fallback in case the response is not valid JSON
        print("Error parsing JSON response from LLM")
        print(f"Response content: {response.content}")
        new_state["tech_stack"] = "Tech stack analysis failed: Could not parse LLM response"
        new_state["framework"] = "unknown"
        new_state["framework_cmd"] = 'CMD ["python", "app.py"]'
        
    # Add messages to history
    new_state["messages"].append(HumanMessage(content="Analyze the tech stack for this project."))
    new_state["messages"].append(AIMessage(content=response.content))
    
    return new_state

# Node 3: Generate Dockerfile
def generate_dockerfile(state: Dict) -> Dict:
    """Generates a Dockerfile based on the tech stack analysis."""
    # Make a copy of the state
    new_state = state.copy()
    
    llm = ChatOpenAI(model_name="gpt-4")
    
    prompt = f"""
    You are a Docker specialist.
    Your job is to create a Dockerfile based on tech stack analysis.

    **Detected Tech Stack:**  
    {new_state["tech_stack"]}

    **Detected Framework:**
    {new_state["framework"]}

    **Recommended Command:**  
    {new_state["framework_cmd"]}

    Based on this information, generate a valid Dockerfile for the project.
    Your response should **only** be a valid Dockerfile with no explanations or markdown formatting.

    **Project Structure:**  
    {new_state["tree_structure"]}
    """
    
    messages = [HumanMessage(content=prompt)]
    response = llm.invoke(messages)
    
    # Update state with the generated Dockerfile
    new_state["docker_file"] = response.content
    new_state["messages"].append(HumanMessage(content="Create a Dockerfile for the given project."))
    new_state["messages"].append(AIMessage(content=response.content))
    
    return new_state

# Define the workflow graph
def create_workflow(project_path: str):
    # Initialize state
    initial_state = create_initial_state(project_path)
    
    # Create graph
    workflow = StateGraph(Dict)
    
    # Add nodes
    workflow.add_node("get_project_structure", get_project_structure)
    workflow.add_node("analyze_tech_stack", analyze_tech_stack)
    workflow.add_node("generate_dockerfile", generate_dockerfile)
    
    # Set the entry point
    workflow.set_entry_point("get_project_structure")
    
    # Connect the nodes
    workflow.add_edge("get_project_structure", "analyze_tech_stack")
    workflow.add_edge("analyze_tech_stack", "generate_dockerfile")
    workflow.add_edge("generate_dockerfile", END)
    
    # Compile the graph
    return workflow.compile(), initial_state

# Main execution
if __name__ == "__main__":
    project_path = r"C:\Users\vasanth\Desktop\ai_deployer_project\test-repos\fastapi"
    
    # Create workflow and initial state
    workflow, initial_state = create_workflow(project_path)
    
    # Execute the workflow
    result = workflow.invoke(initial_state)
    
    # Print the generated Dockerfile
    print("\n=== Generated Dockerfile ===\n")
    print(result["docker_file"])